https://algotrading101.com/learn/interactive-brokers-python-api-native-guide/
https://algotrading101.com/learn/ib_insync-interactive-brokers-api-guide/
https://interactivebrokers.github.io/#


In [12]:
import sys

sys.path.append(r'C:\Users\jimhu\option_trader\src\IBPKG')
sys.path.append(r'\Users\jimhu\option_trader\src')
from ib_insync import *
from option_trader.settings import ib_config as ic
from option_trader.admin import quote
from option_trader.settings import app_settings    
from option_trader.utils.data_getter_ib import IBClient

import pandas as pd
from datetime import time, date, datetime, timedelta
from yahoo_fin import options
from yahoo_fin import stock_info as si
from pytz import timezone

import numpy as np
import json

TWS_LIVE_PORT  = 7496
TWS_PAPER_PORT = 7497
GATEWAY_LIVE_PORT   = 4001
GATEWAY_PAPAER_PORT = 4002

port =  GATEWAY_PAPAER_PORT 
host = '127.0.0.1'
clientID = 1234

symbol_list = ['QQQ', 'AAPL', 'MSFT']

DTE_list = [7, 12, 19, 26]

today = datetime.now(timezone(app_settings.TIMEZONE))

df = pd.DataFrame()
    
with IBClient(host, TWS=True, Live=True) as ibClient:
    for symbol in symbol_list:
        x = ibClient.ib.reqMatchingSymbols(symbol)

        conId = x[0].contract.conId

        x = ibClient.ib.reqSecDefOptParams(symbol, "", "STK", conId)
        
        stock_price = si.get_live_price(symbol)

        OTM_strike = list(filter(lambda x: x >= stock_price, x[0].strikes))     
        OTM_strike = OTM_strike[0]
        OTM_strike = OTM_strike // 5 * 5
        
        print(symbol, stock_price, OTM_strike)

        exp_date_list = []

        #DTElist = [(pd.Timestamp(x).tz_localize(timezone(app_settings.TIMEZONE))-today).days for x in x[0].expirations]

        #print(DTElist)
        
        for DTE in DTE_list: 
            exp_date_target = list(filter(lambda x: (pd.Timestamp(x).tz_localize(timezone(app_settings.TIMEZONE))-today).days - DTE >= 0, x[0].expirations))    
            #exp_date_target = [datetime.strptime(x, '%B %d, %Y').strftime('%Y%m%d') for x in exp_date_target]
            exp_date_list.append(exp_date_target[0])

        print(exp_date_list)

        ibClient.ib.reqMarketDataType(3)
        
        for exp_date in exp_date_list:            
            print(exp_date, OTM_strike)
            contract = Contract()
            contract.symbol = symbol
            contract.secType = 'OPT'
            contract.exchange = 'SMART'
            contract.currency = 'USD'
            contract.lastTradeDateOrContractMonth = exp_date
            contract.strike = OTM_strike
            contract.right = 'C'
            contract.multiplier = '100'
            ibClient.ib.qualifyContracts(contract) 
            x = ibClient.ib.reqMktData(contract, '', False, False, [])
            #print(x)
            ibClient.ib.sleep(5)                     
            ne = pd.DataFrame.from_records(quote.option_chain_rec)
            ne.exp_date = exp_date
            ne.strike = OTM_strike    
            ne.bid =  x.bid
            ne.bidSize = x.bidSize
            ne.ask = x.ask
            ne.askSize = x.askSize
            ne.lastPrice = x.last
            ne.lastSize = x.lastSize            
            ne.open = x.open
            ne.high = x.high
            ne.low = x.low
            ne.close = x.close
            ne.volume = x.volume
            
            m = x.modelGreeks       
            b = x.bidGreeks 
            a = x.askGreeks        
            l = x.lastGreeks        
            print(a, b, l, m)

            ne.impliedVolatility = json.dumps({'model':m.impliedVol if m!= None else None,
                                   'bid':b.impliedVol if b != None else None,
                                   'ask':a.impliedVol if a != None else None,
                                   'last':l.impliedVol if l != None else None})                 
            ne.delta = json.dumps({'model':m.delta if m!= None else None,
                                   'bid':b.delta if b != None else None,
                                   'ask':a.delta if a != None else None,
                                   'last':l.delta if l != None else None})   
            ne.gamma = json.dumps({'model':m.gamma if m != None else None, 
                                   'bid':b.gamma if b != None else None,
                                   'ask':a.gamma if a != None else None,
                                   'last':l.gamma if l != None else None})                
            ne.vega = json.dumps({'model':m.vega if m != None else None, 
                                   'bid':b.vega if b != None else None,
                                   'ask':a.vega if a != None else None,
                                   'last':l.vega if l != None else None})                    
            ne.theta = json.dumps({'model':m.theta if m != None else None, 
                                   'bid':b.theta if b != None else None,
                                   'ask':a.theta if a != None else None,
                                   'last':l.theta if l != None else None})                  

            ne['symbol'] = symbol        
            df = pd.concat([df, ne])  
 
        
        #print(x[0].expirations)

        #print(x[0].strikes)
        
df[['symbol', 'exp_date',  'strike', 'impliedVolatility', 'delta', 'gamma', 'vega', 'theta', 'bid', 'bidSize', 'ask', 'askSize', 'lastPrice', 'volume']]
    

QQQ 398.6700134277344 395.0
['20231220', '20231226', '20240105', '20240112']
20231220 395.0


Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Peer closed connection.


ConnectionError: Socket disconnect

In [ ]:
df[['symbol', 'exp_date',  'strike', 'impliedVolatility', 'delta', 'gamma', 'vega', 'theta', 'bid', 'bidSize', 'ask', 'askSize', 'lastPrice', 'volume']]
    

In [12]:
import pandas as pd
from datetime import time, date, datetime, timedelta
symbol = 'QQQ'
from yahoo_fin import options
from yahoo_fin import stock_info as si
from pytz import timezone

DTE_list = [5, 10, 15, 30]

today = datetime.now(timezone(app_settings.TIMEZONE))
xlist = options.get_expiration_dates(symbol)
#DTE_list = [(pd.Timestamp(x).tz_localize(timezone(app_settings.TIMEZONE))-today).days for x in xlist]
#exp_date_list = [datetime.strptime(x, '%B %d, %Y').strftime('%Y%m%d') for x in xlist]

exp_date_list = []

for DTE in DTE_list: 
    exp_date_target = list(filter(lambda x: (pd.Timestamp(x).tz_localize(timezone(app_settings.TIMEZONE))-today).days - DTE >= 0, xlist))    
    exp_date_target = [datetime.strptime(x, '%B %d, %Y').strftime('%Y%m%d') for x in exp_date_target]
    exp_date_list.append(exp_date_target[0])
    
print(exp_date_list)


stock_price = si.get_live_price(symbol)
strike = stock_price // 5 * 5


#print(stock_price, strike)
    
#for exp_date in exp_date_list:
#    days_to_expire = (pd.Timestamp(exp_date).tz_localize(timezone(app_settings.TIMEZONE))-today).days
#    print(exp_date, days_to_expire)
    

['20231218', '20231222', '20231229', '20240112']


In [27]:
from ib_insync import *
import numpy as np
import pandas as pd
from datetime import time, date, datetime, timedelta
from pytz import timezone

option_chain_rec = {'exp_date':[''],'strike':[np.nan],
                'bid':[np.nan],  'bidSize':[np.nan], 
                'ask':[np.nan],  'askSize':[np.nan],                     
                'lastPrice':[np.nan], 'lastSize':[np.nan],
                'volume':[np.nan],
                'open':[np.nan], 'high':[np.nan], 'low':[np.nan], 'close':[np.nan],                 
                'impliedVolatility':[np.nan], 'delta':[np.nan], 'gamma':[np.nan],
                'vega':[np.nan], 'theta':[np.nan]}

def get_option_chain(symbol, stock_price, strike_limit_pert =0.02, min_days_to_expire=15, max_days_to_expire=30):
        
    trade_date = datetime.now(timezone('US/Eastern'))    
    
    util.startLoop()  # uncomment this line when in a notebook

    ib = IB()

    ib.connect(host, port, clientId=9003)

    x = ib.reqMatchingSymbols(symbol)
    
    conId = x[0].contract.conId
        
    x = ib.reqSecDefOptParams(symbol, "", "STK", conId)
    
    exp_date_list = list(filter(lambda x: (pd.Timestamp(x).tz_localize(timezone('US/Eastern'))-trade_date).days < max_days_to_expire, x[0].expirations))   
   
    exp_date_list=['20231222']
    
    if strike_limit_pert > 0:    
        strikes = list(filter(lambda x: x >= stock_price * (1-strike_limit_pert) and x <= stock_price * (1+strike_limit_pert), x[0].strikes))   
    else:
        strikes = list(filter(lambda x: x >= stock_price, x[0].strikes))     
        strikes = [strikes[0]]
 
    print(exp_date_list)
    
    print(strikes)
    
    #df_new_row = pd.DataFrame.from_records(self.empty_rec)

    ib.reqMarketDataType(3)
        
    df = pd.DataFrame()
    for exp_date in exp_date_list:
        for strike in strikes:        
            print(exp_date, strike)
            contract = Contract()
            contract.symbol = symbol
            contract.secType = 'OPT'
            contract.exchange = 'SMART'
            contract.currency = 'USD'
            contract.lastTradeDateOrContractMonth = exp_date
            contract.strike = strike
            contract.right = 'C'
            contract.multiplier = '100'
            ib.qualifyContracts(contract) 
            x = ib.reqMktData(contract, '', False, False, [])
            ib.sleep(5)            
            print(x)
            print(x.bid, x.ask, x.last)            
            ne = pd.DataFrame.from_records(option_chain_rec)
            ne.exp_date = exp_date
            ne.strike = strike    
            ne.bid =  x.bid
            ne.bidSize = x.bidSize
            ne.ask = x.ask
            ne.askSize = x.askSize
            ne.lastPrice = x.last
            print(ne.lastPrice, x.last)
            ne.lastSize = x.lastSize            
            ne.open = x.open
            ne.high = x.high
            ne.low = x.low
            ne.close = x.close

            m = x.modelGreeks       
            b = x.bidGreeks 
            a = x.askGreeks             
            if m != None:
                #print(m)
                ne.impliedVolatility = m.impliedVol
                ne.delta = m.delta
                ne.gamma = m.gamma
                ne.vega =  m.vega
                ne.theta = m.theta
                #ne.sigma = m.sigma
                ne.volume = x.volume

            print(ne)
            df = pd.concat([df, ne])  
                        
    ib.disconnect()
    
    return df

df = get_option_chain('AAPL', si.get_live_price('AAPL'), strike_limit_pert=0)

df

['20231222']
[197.5]
20231222 197.5


Error 354, reqId 6: Requested market data is not subscribed.AAPL DEC 22 '23 197.5 Call (AAPL  231222C00197500) /TOP/ALL, contract: Contract(secType='OPT', conId=669475750, symbol='AAPL', lastTradeDateOrContractMonth='20231222', strike=197.5, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='AAPL  231222C00197500', tradingClass='AAPL')
Error 10090, reqId 6: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.AAPL NASDAQ.NMS/TOP/ALL, contract: Contract(secType='OPT', conId=669475750, symbol='AAPL', lastTradeDateOrContractMonth='20231222', strike=197.5, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='AAPL  231222C00197500', tradingClass='AAPL')


Ticker(contract=Contract(secType='OPT', conId=669475750, symbol='AAPL', lastTradeDateOrContractMonth='20231222', strike=197.5, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='AAPL  231222C00197500', tradingClass='AAPL'), time=datetime.datetime(2023, 12, 11, 13, 56, 46, 121361, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.01, modelGreeks=OptionComputation(tickAttrib=0, impliedVol=0.15044448123092904, delta=0.0, optPrice=0.0, pvDividend=0.0, gamma=0.0, vega=0.0, theta=0.0, undPrice=0.0), bboExchange='c70003', snapshotPermissions=1)
nan nan nan
0   NaN
Name: lastPrice, dtype: float64 nan
   ask  askSize  bid  bidSize  close  delta  exp_date  gamma  high  \
0  NaN      NaN  NaN      NaN    NaN    0.0  20231222    0.0   NaN   

   impliedVolatility  lastPrice  lastSize  low  open  strike  theta  vega  \
0           0.150444        NaN       NaN  NaN   NaN   197.5    0.0   0.0   

   volume  
0     NaN  


,ask,askSize,bid,bidSize,close,delta,exp_date,gamma,high,impliedVolatility,lastPrice,lastSize,low,open,strike,theta,vega,volume
0,NaN,NaN,NaN,NaN,NaN,0.0,20231222,0.0,NaN,0.150444,NaN,NaN,NaN,NaN,197.5,0.0,0.0,NaN


In [ ]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper  

class IBapi(EWrapper, EClient):
     def __init__(self):
         EClient.__init__(self, self) 

app = IBapi()
app.connect(host, port, clientID)
app.run()


#Uncomment this section if unable to connect
#and to prevent errors on a reconnect
#import time
#time.sleep(2)
#app.disconnect()

ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2107 HMDS data farm connection is inactive but should be available upon demand.ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


In [2]:
from ibapi.ticktype import TickTypeEnum

for i in range(91):
	print(TickTypeEnum.to_str(i), i)

BID_SIZE 0
BID 1
ASK 2
ASK_SIZE 3
LAST 4
LAST_SIZE 5
HIGH 6
LOW 7
VOLUME 8
CLOSE 9
BID_OPTION_COMPUTATION 10
ASK_OPTION_COMPUTATION 11
LAST_OPTION_COMPUTATION 12
MODEL_OPTION 13
OPEN 14
LOW_13_WEEK 15
HIGH_13_WEEK 16
LOW_26_WEEK 17
HIGH_26_WEEK 18
LOW_52_WEEK 19
HIGH_52_WEEK 20
AVG_VOLUME 21
OPEN_INTEREST 22
OPTION_HISTORICAL_VOL 23
OPTION_IMPLIED_VOL 24
OPTION_BID_EXCH 25
OPTION_ASK_EXCH 26
OPTION_CALL_OPEN_INTEREST 27
OPTION_PUT_OPEN_INTEREST 28
OPTION_CALL_VOLUME 29
OPTION_PUT_VOLUME 30
INDEX_FUTURE_PREMIUM 31
BID_EXCH 32
ASK_EXCH 33
AUCTION_VOLUME 34
AUCTION_PRICE 35
AUCTION_IMBALANCE 36
MARK_PRICE 37
BID_EFP_COMPUTATION 38
ASK_EFP_COMPUTATION 39
LAST_EFP_COMPUTATION 40
OPEN_EFP_COMPUTATION 41
HIGH_EFP_COMPUTATION 42
LOW_EFP_COMPUTATION 43
CLOSE_EFP_COMPUTATION 44
LAST_TIMESTAMP 45
SHORTABLE 46
FUNDAMENTAL_RATIOS 47
RT_VOLUME 48
HALTED 49
BID_YIELD 50
ASK_YIELD 51
LAST_YIELD 52
CUST_OPTION_COMPUTATION 53
TRADE_COUNT 54
TRADE_RATE 55
VOLUME_RATE 56
LAST_RTH_TRADE 57
RT_HISTORICAL_VO

In [6]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract

import threading
import time

class IBapi(EWrapper, EClient):
	def __init__(self):
		EClient.__init__(self, self)
	def tickPrice(self, reqId, tickType, price, attrib):
		if tickType == 2 and reqId == 1:
			print('The current ask price is: ', price)

def run_loop():
	app.run()

app = IBapi()
app.connect(host, port, 123)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

#Create contract object
apple_contract = Contract()
apple_contract.symbol = 'AAPL'
apple_contract.secType = 'STK'
apple_contract.exchange = 'SMART'
apple_contract.currency = 'USD'

contract = Contract()
contract.symbol = "AAPL"
contract.secType = "OPT"
contract.exchange = "SMART"
contract.currency = "USD"
contract.lastTradeDateOrContractMonth = "20231215"
contract.strike = 190
contract.right = "C"
contract.multiplier = "100"

#Request Market Data
#app.reqMktData(1, apple_contract, '', False, False, [])
app.reqMarketDataType(3)
app.reqMktData(3, contract, '', False, False, [])
time.sleep(10) #Sleep interval to allow time for incoming price data
app.disconnect()


ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2107 HMDS data farm connection is inactive but should be available upon demand.ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil
ERROR 3 354 Requested market data is not subscribed.AAPL DEC 15 '23 190 Call (AAPL  231215C00190000) /TOP/ALL
ERROR 3 10167 Requested market data is not subscribed. Displaying delayed market data.
ERROR 3 10090 Part of requested market data is not subscribed. Subscription-independent ticks are still active.AAPL NASDAQ.NMS/TOP/ALL
ERROR 3 10197 No market data during competing live session


None


In [ ]:
#https://algotrading101.com/learn/interactive-brokers-python-api-native-guide/
#https://algotrading101.com/learn/ib_insync-interactive-brokers-api-guide/
#https://interactivebrokers.github.io/#

from ibapi.client import *
from ibapi.wrapper import *

class TestApp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)

    def nextValidId(self, orderId: int):
        
        mycontract = Contract()
        mycontract.symbol = "AAPL"
        mycontract.secType = "STK"
        mycontract.exchange = "SMART"
        mycontract.currency = "USD"
        
        contract = Contract()
        contract.symbol = "AAPL"
        contract.secType = "OPT"
        contract.exchange = "SMART"
        contract.currency = "USD"
        contract.lastTradeDateOrContractMonth = "20231215"
        contract.strike = 190
        contract.right = "C"
        contract.multiplier = "100"

        self.reqMarketDataType(3)
        
        self.reqSecDefOptParams(0,"AAPL", "STK", "", 265598)        
        
        self.reqHistoricalData(orderId, contract, "20231208 09:30:00 US/Eastern", "1 D", "1 hour", "TRADES", 0, 1, 0, [])

    def historicalData(self, reqId, bar):
        print(f"Historical Data: {bar}")

    def historicalDataEnd(self, reqId, start, end):
        print(f"End of HistoricalData")
        print(f"Start: {start}, End: {end}")


app = TestApp()
app.connect(host, port, clientID)
app.run()

ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2107 HMDS data farm connection is inactive but should be available upon demand.ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil
ERROR 0 321 Error validating request.-'cB' : cause - Invalid security type - 
ERROR -1 2106 HMDS data farm connection is OK:ushmds


Historical Data: Date: 20231207 09:30:00 US/Eastern, Open: 4.65, High: 5.7, Low: 4.6, Close: 5.15, Volume: 559, WAP: 5.265, BarCount: 153
Historical Data: Date: 20231207 10:00:00 US/Eastern, Open: 4.9, High: 5.8, Low: 4.9, Close: 5.62, Volume: 799, WAP: 5.578, BarCount: 113
Historical Data: Date: 20231207 11:00:00 US/Eastern, Open: 5.6, High: 5.65, Low: 5, Close: 5.65, Volume: 145, WAP: 5.514, BarCount: 49
Historical Data: Date: 20231207 12:00:00 US/Eastern, Open: 5.7, High: 5.8, Low: 5.47, Close: 5.5, Volume: 61, WAP: 5.681, BarCount: 31
Historical Data: Date: 20231207 13:00:00 US/Eastern, Open: 5.35, High: 5.55, Low: 5.2, Close: 5.25, Volume: 284, WAP: 5.441, BarCount: 38
Historical Data: Date: 20231207 14:00:00 US/Eastern, Open: 5.15, High: 5.35, Low: 5.05, Close: 5.11, Volume: 189, WAP: 5.199, BarCount: 27
Historical Data: Date: 20231207 15:00:00 US/Eastern, Open: 5.1, High: 5.25, Low: 5, Close: 5.1, Volume: 238, WAP: 5.103, BarCount: 42
End of HistoricalData
Start: 20231207 09:30:

ERROR -1 2107 HMDS data farm connection is inactive but should be available upon demand.ushmds
ERROR -1 1100 Connectivity between IBKR and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IBKR and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IBKR and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IBKR and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IBKR and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IBKR and Trader Workstation has been lost.
ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 2157 Sec-def data farm connection is broken:secdefil
ERROR -1 2158 Sec-def data farm connection is OK:secdefil
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 1102 Connectivity between IBKR and Trader Workstation has been restored - data maintained. The following farms are connected: usfarm; secdefil. The following farms are not connected: ushmds.
ERR

In [ ]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract

import threading
import time

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.reqMarketDataType(3)
        
    def tickPrice(self, reqId, tickType, price, attrib):
        #if tickType == 2 and reqId == 1:       
        print('The current ask price is: ', price)

def run_loop():
    app.run()

app = IBapi()
app.connect(host, port, 123)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

#Create contract object
apple_contract = Contract()
apple_contract.symbol = 'MSFT'
apple_contract.secType = 'STK'
apple_contract.exchange = 'SMART'
apple_contract.currency = 'USD'

#Request Market Data
app.reqMarketDataType(3)
app.reqMktData(1, apple_contract, '', False, False, [])

time.sleep(10) #Sleep interval to allow time for incoming price data
app.disconnect()

In [2]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
#from ibapi.contract import Contract
from ibapi.common import SetOfString
from ibapi.common import SetOfFloat

import threading
import time

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        
        
    def securityDefinitionOptionParameter(self, 
                                          reqId:int, 
                                          exchange:str, 
                                          underlyingConId:int, 
                                          tradingClass:str, 
                                          multiplier:str, 
                                          expirations:SetOfString, 
                                          strikes:SetOfFloat):
        print("SecurityDefinitionOptionParameter.", "ReqId:", 
              reqId, "Exchange:", 
              exchange, "Underlying conId:", 
              underlyingConId, "TradingClass:", 
              tradingClass, "Multiplier:", 
              multiplier, "Expirations:", 
              expirations, "Strikes:", 
              str(strikes),"\n")

    def securityDefinitionOptionParameterEnd(self, reqId:int):

        print("SecurityDefinitionOptionParameterEnd. ReqId:", reqId)

def run_loop():
    app.run()

app = IBapi()
app.connect(host, port, cliendID)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

app.reqSecDefOptParams(0,"MSFT", "", "STK", 272093)

time.sleep(10) #Sleep interval to allow time for incoming price data
app.disconnect()

ERROR -1 2110 Connectivity between Trader Workstation and server is broken. It will be restored automatically.
ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 2105 HMDS data farm connection is broken:ushmds
ERROR -1 2157 Sec-def data farm connection is broken:secdefnj
ERROR 0 322 Error processing request. Sending message failed


In [4]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract, ContractDetails
from ibapi.common import SetOfString
from ibapi.common import SetOfFloat

import threading
import time

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        
        
    def contractDetails(self, reqId: int, contractDetails: ContractDetails):
        super().contractDetails(reqId, contractDetails)
        print('========================')
        print(contractDetails)
        print('-------------------------')
        #printinstance(contractDetails)

    def contractDetailsEnd(self, reqId: int):
        super().contractDetailsEnd(reqId)
        print("ContractDetailsEnd. ReqId:", reqId)

def run_loop():
    app.run()

app = IBapi()
app.connect(host, port, 123)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

contract = Contract()
contract.symbol = 'TSLA'
contract.secType = "OPT"
contract.exchange = "SMART"
contract.currency = "USD"
      
x = app.reqContractDetails(3, contract)
time.sleep(80) #Sleep interval to allow time for incoming price data
print(x)
app.disconnect()

ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2107 HMDS data farm connection is inactive but should be available upon demand.ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


514123552,TSLA,OPT,20240119,210,C,100,SMART,,USD,TSLA  240119C00210000,TSLA,False,,,,combo:,TSLA,0.01,ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,COND,CONDORDER,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,FOK,GAT,GTC,GTD,GTT,HID,ICE,IOC,LIT,LMT,MIT,MKT,MTL,NGCOMB,NONALGO,OCA,OPENCLOSE,PAON,PEGMIDVOL,PEGMKTVOL,PEGPRMVOL,PEGSRFVOL,POSTONLY,PRICECHK,REL,RELPCTOFS,RELSTK,SCALE,SCALERST,SIZECHK,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,VOLAT,WHATIF,SMART,AMEX,CBOE,PHLX,PSE,ISE,BOX,BATS,NASDAQOM,CBOE2,NASDAQBX,MIAX,GEMINI,EDGX,MERCURY,PEARL,EMERALD,IBUSOPT,1,76792991,TESLA INC,202401,,,,US/Eastern,20231209:CLOSED;20231210:CLOSED;20231211:0930-20231211:1600;20231212:0930-20231212:1600;20231213:0930-20231213:1600;20231214:0930-20231214:1600,20231209:CLOSED;20231210:CLOSED;20231211:0930-20231211:1600;20231212:0930-20231212:1600;20231213:0930-20231213:1600;20231214:0930-20231214:1600,,0,TSLA,STK,32,109,109,109,109,109,109,109,32,109,32,109,109,109,109,109,109,32,2,None,

514123738,TSLA,OPT,20240119,236.67,C,100,SMART,,USD,TSLA  240119C00236670,TSLA,False,,,,combo:,TSLA,0.01,ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,COND,CONDORDER,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,FOK,GAT,GTC,GTD,GTT,HID,ICE,IOC,LIT,LMT,MIT,MKT,MTL,NGCOMB,NONALGO,OCA,OPENCLOSE,PAON,PEGMIDVOL,PEGMKTVOL,PEGPRMVOL,PEGSRFVOL,POSTONLY,PRICECHK,REL,RELPCTOFS,RELSTK,SCALE,SCALERST,SIZECHK,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,VOLAT,WHATIF,SMART,AMEX,CBOE,PHLX,PSE,ISE,BOX,BATS,NASDAQOM,CBOE2,NASDAQBX,MIAX,GEMINI,EDGX,MERCURY,PEARL,EMERALD,IBUSOPT,1,76792991,TESLA INC,202401,,,,US/Eastern,20231209:CLOSED;20231210:CLOSED;20231211:0930-20231211:1600;20231212:0930-20231212:1600;20231213:0930-20231213:1600;20231214:0930-20231214:1600,20231209:CLOSED;20231210:CLOSED;20231211:0930-20231211:1600;20231212:0930-20231212:1600;20231213:0930-20231213:1600;20231214:0930-20231214:1600,,0,TSLA,STK,32,109,109,109,109,109,109,109,32,109,32,109,109,109,109,109,109,32,2,No

514123852,TSLA,OPT,20240119,263.33,C,100,SMART,,USD,TSLA  240119C00263330,TSLA,False,,,,combo:,TSLA,0.01,ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,COND,CONDORDER,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,FOK,GAT,GTC,GTD,GTT,HID,ICE,IOC,LIT,LMT,MIT,MKT,MTL,NGCOMB,NONALGO,OCA,OPENCLOSE,PAON,PEGMIDVOL,PEGMKTVOL,PEGPRMVOL,PEGSRFVOL,POSTONLY,PRICECHK,REL,RELPCTOFS,RELSTK,SCALE,SCALERST,SIZECHK,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,VOLAT,WHATIF,SMART,AMEX,CBOE,PHLX,PSE,ISE,BOX,BATS,NASDAQOM,CBOE2,NASDAQBX,MIAX,GEMINI,EDGX,MERCURY,PEARL,EMERALD,IBUSOPT,1,76792991,TESLA INC,202401,,,,US/Eastern,20231209:CLOSED;20231210:CLOSED;20231211:0930-20231211:1600;20231212:0930-20231212:1600;20231213:0930-20231213:1600;20231214:0930-20231214:1600,20231209:CLOSED;20231210:CLOSED;20231211:0930-20231211:1600;20231212:0930-20231212:1600;20231213:0930-20231213:1600;20231214:0930-20231214:1600,,0,TSLA,STK,32,109,109,109,109,109,109,109,32,109,32,109,109,109,109,109,109,32,2,No

ERROR -1 1100 Connectivity between IBKR and Trader Workstation has been lost.
ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 2157 Sec-def data farm connection is broken:secdefil
ERROR -1 2158 Sec-def data farm connection is OK:secdefil
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 1102 Connectivity between IBKR and Trader Workstation has been restored - data maintained. The following farms are connected: usfarm; secdefil. The following farms are not connected: ushmds.


None


In [6]:
from ibapi.ticktype import TickTypeEnum

for i in range(91):
	print(TickTypeEnum.to_str(i), i)

BID_SIZE 0
BID 1
ASK 2
ASK_SIZE 3
LAST 4
LAST_SIZE 5
HIGH 6
LOW 7
VOLUME 8
CLOSE 9
BID_OPTION_COMPUTATION 10
ASK_OPTION_COMPUTATION 11
LAST_OPTION_COMPUTATION 12
MODEL_OPTION 13
OPEN 14
LOW_13_WEEK 15
HIGH_13_WEEK 16
LOW_26_WEEK 17
HIGH_26_WEEK 18
LOW_52_WEEK 19
HIGH_52_WEEK 20
AVG_VOLUME 21
OPEN_INTEREST 22
OPTION_HISTORICAL_VOL 23
OPTION_IMPLIED_VOL 24
OPTION_BID_EXCH 25
OPTION_ASK_EXCH 26
OPTION_CALL_OPEN_INTEREST 27
OPTION_PUT_OPEN_INTEREST 28
OPTION_CALL_VOLUME 29
OPTION_PUT_VOLUME 30
INDEX_FUTURE_PREMIUM 31
BID_EXCH 32
ASK_EXCH 33
AUCTION_VOLUME 34
AUCTION_PRICE 35
AUCTION_IMBALANCE 36
MARK_PRICE 37
BID_EFP_COMPUTATION 38
ASK_EFP_COMPUTATION 39
LAST_EFP_COMPUTATION 40
OPEN_EFP_COMPUTATION 41
HIGH_EFP_COMPUTATION 42
LOW_EFP_COMPUTATION 43
CLOSE_EFP_COMPUTATION 44
LAST_TIMESTAMP 45
SHORTABLE 46
FUNDAMENTAL_RATIOS 47
RT_VOLUME 48
HALTED 49
BID_YIELD 50
ASK_YIELD 51
LAST_YIELD 52
CUST_OPTION_COMPUTATION 53
TRADE_COUNT 54
TRADE_RATE 55
VOLUME_RATE 56
LAST_RTH_TRADE 57
RT_HISTORICAL_VO

In [7]:

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from threading import Timer

class TestApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)

    #def error(self, reqId, errorCode, errorString):
    #    print("Error: ", reqId, " ", errorCode, " ", errorString)

    def nextValidId(self, orderId):
        self.start()

    def updatePortfolio(self, contract: Contract, position: float, marketPrice: float, marketValue: float,
                        averageCost: float, unrealizedPNL: float, realizedPNL: float, accountName: str):
        print("UpdatePortfolio.", "Symbol:", contract.symbol, "SecType:", contract.secType, "Exchange:", contract.exchange,
              "Position:", position, "MarketPrice:", marketPrice, "MarketValue:", marketValue, "AverageCost:", averageCost,
              "UnrealizedPNL:", unrealizedPNL, "RealizedPNL:", realizedPNL, "AccountName:", accountName)

    def updateAccountValue(self, key: str, val: str, currency: str, accountName: str):
        print("UpdateAccountValue. Key:", key, "Value:", val, "Currency:", currency, "AccountName:", accountName)

    def updateAccountTime(self, timeStamp: str):
        print("UpdateAccountTime. Time:", timeStamp)

    def accountDownloadEnd(self, accountName: str):
        print("AccountDownloadEnd. Account:", accountName)

    def start(self):
        # Account number can be omitted when using reqAccountUpdates with single account structure
        self.reqAccountUpdates(True, "")

    def stop(self):
        self.reqAccountUpdates(False, "")
        self.done = True
        self.disconnect()

def main():
    app = TestApp()
    app.connect(host, port, 0)

    Timer(5, app.stop).start()
    app.run()

if __name__ == "__main__":
    main()


ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:eufarmnj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj
ERROR 2147483647 321 Error validating request.-'bv' : cause - The account code is required for this operation.


In [ ]:
class TestApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)

    def error(self, reqId, errorCode, errorString):
        print("Error: ", reqId, " ", errorCode, " ", errorString)

    def nextValidId(self, orderId):
        self.start()

    def updatePortfolio(self, contract: Contract, position: float, marketPrice: float, marketValue: float,
                        averageCost: float, unrealizedPNL: float, realizedPNL: float, accountName: str):
        print("UpdatePortfolio.", "Symbol:", contract.symbol, "SecType:", contract.secType, "Exchange:", contract.exchange,
              "Position:", position, "MarketPrice:", marketPrice, "MarketValue:", marketValue, "AverageCost:", averageCost,
              "UnrealizedPNL:", unrealizedPNL, "RealizedPNL:", realizedPNL, "AccountName:", accountName)

    def updateAccountValue(self, key: str, val: str, currency: str, accountName: str):
        print("UpdateAccountValue. Key:", key, "Value:", val, "Currency:", currency, "AccountName:", accountName)

    def updateAccountTime(self, timeStamp: str):
        print("UpdateAccountTime. Time:", timeStamp)

    def accountDownloadEnd(self, accountName: str):
        print("AccountDownloadEnd. Account:", accountName)

    def start(self):
        # Account number can be omitted when using reqAccountUpdates with single account structure
        self.reqAccountUpdates(True, "")

    def stop(self):
        self.reqAccountUpdates(False, "")
        self.done = True
        self.disconnect()


In [6]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract

import threading
import time

class IBapi(EWrapper, EClient):
	def __init__(self):
		EClient.__init__(self, self)
	def historicalData(self, reqId, bar):
		print(f'Time: {bar.date} Close: {bar.close}')
		
def run_loop():
	app.run()

app = IBapi()
app.connect(host, port, 123)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

#Create contract object
eurusd_contract = Contract()
eurusd_contract.symbol = 'EUR'
eurusd_contract.secType = 'CASH'
eurusd_contract.exchange = 'IDEALPRO'
eurusd_contract.currency = 'USD'

#Request historical candles
app.reqHistoricalData(1, eurusd_contract, '', '2 D', '1 hour', 'BID', 0, 2, False, [])

time.sleep(5) #sleep to allow enough time for data to be returned
app.disconnect()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:eufarmnj
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj
ERROR -1 2106 HMDS data farm connection is OK:cashhmds


Time: 1700604900 Close: 1.09109
Time: 1700607600 Close: 1.09171
Time: 1700611200 Close: 1.09196
Time: 1700614800 Close: 1.09133
Time: 1700618400 Close: 1.09159
Time: 1700622000 Close: 1.09144
Time: 1700625600 Close: 1.09137
Time: 1700629200 Close: 1.09073
Time: 1700632800 Close: 1.09003
Time: 1700636400 Close: 1.09146
Time: 1700640000 Close: 1.08854
Time: 1700643600 Close: 1.09025
Time: 1700647200 Close: 1.09007
Time: 1700650800 Close: 1.08999
Time: 1700654400 Close: 1.09066
Time: 1700658000 Close: 1.0896
Time: 1700661600 Close: 1.08742
Time: 1700665200 Close: 1.08657
Time: 1700668800 Close: 1.08703
Time: 1700672400 Close: 1.08786
Time: 1700676000 Close: 1.08846
Time: 1700679600 Close: 1.08838
Time: 1700683200 Close: 1.08874
Time: 1700686800 Close: 1.08881
Time: 1700691300 Close: 1.08861
Time: 1700694000 Close: 1.08878
Time: 1700697600 Close: 1.0892
Time: 1700701200 Close: 1.0895
Time: 1700704800 Close: 1.09035
Time: 1700708400 Close: 1.0905
Time: 1700712000 Close: 1.09026
Time: 170071

In [ ]:
import sys

sys.path.append(r'C:\TWS API\source\pythonclient')

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

import threading
import time

class IBapi(EWrapper, EClient):
	def __init__(self):
		EClient.__init__(self, self)

	def nextValidId(self, orderId: int):
		super().nextValidId(orderId)
		self.nextorderId = orderId
		print('The next valid order id is: ', self.nextorderId)

	def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
		print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining, 'lastFillPrice', lastFillPrice)
	
	def openOrder(self, orderId, contract, order, orderState):
		print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)

	def execDetails(self, reqId, contract, execution):
		print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares, execution.lastLiquidity)


def run_loop():
	app.run()

#Function to create FX Order contract
def FX_order(symbol):
	contract = Contract()
	contract.symbol = symbol[:3]
	contract.secType = 'CASH'
	contract.exchange = 'IDEALPRO'
	contract.currency = symbol[3:]
	return contract

app = IBapi()
app.connect('127.0.0.1', 7497, 123)

app.nextorderId = None

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

#Check if the API is connected via orderid
while True:
	if isinstance(app.nextorderId, int):
		print('connected')
		break
	else:
		print('waiting for connection')
		time.sleep(1)

#Create order object
order = Order()
order.action = 'BUY'
order.totalQuantity = 100000
order.orderType = 'LMT'
order.lmtPrice = '1.10'

#Place order
app.placeOrder(app.nextorderId, FX_order('EURUSD'), order)
#app.nextorderId += 1

time.sleep(3)

#Cancel order 
print('cancelling order')
app.cancelOrder(app.nextorderId, "")

time.sleep(3)
app.disconnect()

In [6]:
from ib_insync import *

util.startLoop()  # uncomment this line when in a notebook

ib = IB()

ib.connect(host, port, clientId=clinetID)

contract = Forex('EURUSD')

x = ib.reqSecDefOptParams("AAPL", "", "STK", 265598)

print(x[0])

ib.disconnect()

# convert to pandas dataframe:
#df = util.df(bars)
#print(df)

Peer closed connection. clientId 4 already in use?
API connection failed: TimeoutError()


TimeoutError: 

In [2]:
x[0].expirations

['20231124',
 '20231201',
 '20231208',
 '20231215',
 '20231222',
 '20231229',
 '20240119',
 '20240216',
 '20240315',
 '20240419',
 '20240621',
 '20240719',
 '20240920',
 '20241220',
 '20250117',
 '20250620',
 '20250919',
 '20251219',
 '20260116']

In [ ]:
from ib_insync import *

try:
    get_ipython    
    util.startLoop()  # uncomment this line when in a notebook
except:
    pass
    
ib = IB()

ib.connect('127.0.0.1', 7497, clientId=3)

In [54]:
from ib_insync import *
import numpy as np
import pandas as pd
from datetime import time, date, datetime, timedelta
from pytz import timezone

option_chain_rec = {'exp_date':[''],'strike':[np.nan],
                'bid':[np.nan],  'bidSize':[np.nan], 
                'ask':[np.nan],  'askSize':[np.nan],                     
                'lastPrice':[np.nan], 'lastSize':[np.nan],
                'volume':[np.nan],
                'open':[np.nan], 'high':[np.nan], 'low':[np.nan], 'close':[np.nan],                 
                'impliedVolatility':[np.nan], 'delta':[np.nan], 'gamma':[np.nan],
                'vega':[np.nan], 'theta':[np.nan]}

def get_option_chain(symbol, stock_price, strike_limit_pert =0.02, min_days_to_expire=15, max_days_to_expire=30):
        
    trade_date = datetime.now(timezone('US/Eastern'))    
    
    util.startLoop()  # uncomment this line when in a notebook

    ib = IB()

    ib.connect(host, port, clientId=12345)

    x = ib.reqMatchingSymbols(symbol)
    
    conId = x[0].contract.conId
        
    x = ib.reqSecDefOptParams(symbol, "", "STK", conId)
    
    exp_date_list = list(filter(lambda x: (pd.Timestamp(x).tz_localize(timezone('US/Eastern'))-trade_date).days < max_days_to_expire, x[0].expirations))   
   
    if strike_limit_pert > 0:    
        strikes = list(filter(lambda x: x >= stock_price * (1-strike_limit_pert) and x <= stock_price * (1+strike_limit_pert), x[0].strikes))   
    else:
        strikes = list(filter(lambda x: x >= stock_price, x[0].strikes))     
        strikes = [strikes[0]]
 
    print(exp_date_list)
    
    print(strikes)
    
    #df_new_row = pd.DataFrame.from_records(self.empty_rec)

    ib.reqMarketDataType(3)
        
    df = pd.DataFrame()
    for exp_date in exp_date_list:
        for strike in strikes:        
            print(exp_date, strike)
            contract = Contract()
            contract.symbol = symbol
            contract.secType = 'OPT'
            contract.exchange = 'SMART'
            contract.currency = 'USD'
            contract.lastTradeDateOrContractMonth = exp_date
            contract.strike = strike
            contract.right = 'C'
            contract.multiplier = '100'
            ib.qualifyContracts(contract) 
            x = ib.reqMktData(contract, '', False, False, [])
            ib.sleep(5)            
            print(x)
            print(x.bid, x.ask, x.last)            
            ne = pd.DataFrame.from_records(option_chain_rec)
            ne.exp_date = exp_date
            ne.strike = strike    
            ne.bid =  x.bid
            ne.bidSize = x.bidSize
            ne.ask = x.ask
            ne.askSize = x.askSize
            ne.lastPrice = x.last
            print(ne.lastPrice, x.last)
            ne.lastSize = x.lastSize            
            ne.open = x.open
            ne.high = x.high
            ne.low = x.low
            ne.close = x.close

            m = x.modelGreeks       
            b = x.bidGreeks 
            a = x.askGreeks             
            if m != None:
                #print(m)
                ne.impliedVolatility = m.impliedVol
                ne.delta = m.delta
                ne.gamma = m.gamma
                ne.vega =  m.vega
                ne.theta = m.theta
                #ne.sigma = m.sigma
                ne.volume = x.volume

            print(ne)
            df = pd.concat([df, ne])  
                        
    ib.disconnect()
    
    return df

df = get_option_chain('MSFT', 374.23, strike_limit_pert=0)

df

['20231215', '20231222', '20231229', '20240105']
[375.0]
20231215 375.0


Error 10090, reqId 6: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.MSFT NASDAQ.NMS/TOP/ALL, contract: Contract(secType='OPT', conId=632234624, symbol='MSFT', lastTradeDateOrContractMonth='20231215', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  231215C00375000', tradingClass='MSFT')


Ticker(contract=Contract(secType='OPT', conId=632234624, symbol='MSFT', lastTradeDateOrContractMonth='20231215', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  231215C00375000', tradingClass='MSFT'), time=datetime.datetime(2023, 12, 10, 15, 14, 4, 157800, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.0, modelGreeks=OptionComputation(tickAttrib=0, impliedVol=0.1852815702270387, delta=0.4849159097275644, optPrice=3.1258953237496243, pvDividend=0.0, gamma=0.048099001444456994, vega=0.17896878285258566, theta=-0.34361349230794425, undPrice=374.3), bboExchange='c70003', snapshotPermissions=1)
nan nan nan
0   NaN
Name: lastPrice, dtype: float64 nan
   ask  askSize  bid  bidSize  close     delta  exp_date     gamma  high  \
0  NaN      NaN  NaN      NaN    NaN  0.484916  20231215  0.048099   NaN   

   impliedVolatility  lastPrice  lastSize  low  open  strike     theta  \
0           0.185282        NaN       NaN  NaN   NaN   375.0

Error 10090, reqId 8: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.MSFT NASDAQ.NMS/TOP/ALL, contract: Contract(secType='OPT', conId=663787506, symbol='MSFT', lastTradeDateOrContractMonth='20231222', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  231222C00375000', tradingClass='MSFT')


Ticker(contract=Contract(secType='OPT', conId=663787506, symbol='MSFT', lastTradeDateOrContractMonth='20231222', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  231222C00375000', tradingClass='MSFT'), time=datetime.datetime(2023, 12, 10, 15, 14, 11, 296224, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.01, bid=-1.0, bidSize=0.0, ask=-1.0, askSize=0.0, last=5.21, lastSize=34.0, volume=1529.0, open=0.0, high=5.35, low=3.3, close=4.5, ticks=[TickData(time=datetime.datetime(2023, 12, 10, 15, 14, 11, 296224, tzinfo=datetime.timezone.utc), tickType=66, price=-1.0, size=0.0), TickData(time=datetime.datetime(2023, 12, 10, 15, 14, 11, 296224, tzinfo=datetime.timezone.utc), tickType=67, price=-1.0, size=0.0)], bidGreeks=OptionComputation(tickAttrib=0, impliedVol=None, delta=None, optPrice=None, pvDividend=0.0, gamma=None, vega=-2.0, theta=-2.0, undPrice=374.3), askGreeks=OptionComputation(tickAttrib=0, impliedVol=None, delta=None, optP

Error 10090, reqId 10: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.MSFT NASDAQ.NMS/TOP/ALL, contract: Contract(secType='OPT', conId=664948989, symbol='MSFT', lastTradeDateOrContractMonth='20231229', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  231229C00375000', tradingClass='MSFT')


Ticker(contract=Contract(secType='OPT', conId=664948989, symbol='MSFT', lastTradeDateOrContractMonth='20231229', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  231229C00375000', tradingClass='MSFT'), marketDataType=3, minTick=0.01, bboExchange='c70003', snapshotPermissions=1)
nan nan nan
0   NaN
Name: lastPrice, dtype: float64 nan
   ask  askSize  bid  bidSize  close  delta  exp_date  gamma  high  \
0  NaN      NaN  NaN      NaN    NaN    NaN  20231229    NaN   NaN   

   impliedVolatility  lastPrice  lastSize  low  open  strike  theta  vega  \
0                NaN        NaN       NaN  NaN   NaN   375.0    NaN   NaN   

   volume  
0     NaN  
20240105 375.0


Error 10090, reqId 12: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.MSFT NASDAQ.NMS/TOP/ALL, contract: Contract(secType='OPT', conId=667689557, symbol='MSFT', lastTradeDateOrContractMonth='20240105', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  240105C00375000', tradingClass='MSFT')


Ticker(contract=Contract(secType='OPT', conId=667689557, symbol='MSFT', lastTradeDateOrContractMonth='20240105', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  240105C00375000', tradingClass='MSFT'), time=datetime.datetime(2023, 12, 10, 15, 14, 17, 704564, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.01, last=7.75, lastSize=1.0, volume=1136.0, high=7.8, low=5.7, close=6.75, ticks=[TickData(time=datetime.datetime(2023, 12, 10, 15, 14, 17, 704564, tzinfo=datetime.timezone.utc), tickType=72, price=7.8, size=0.0), TickData(time=datetime.datetime(2023, 12, 10, 15, 14, 17, 704564, tzinfo=datetime.timezone.utc), tickType=73, price=5.7, size=0.0), TickData(time=datetime.datetime(2023, 12, 10, 15, 14, 17, 704564, tzinfo=datetime.timezone.utc), tickType=74, price=-1.0, size=1136.0), TickData(time=datetime.datetime(2023, 12, 10, 15, 14, 17, 704564, tzinfo=datetime.timezone.utc), tickType=75, price=6.75, size=0.0)], bidGreeks=OptionCom

,ask,askSize,bid,bidSize,close,delta,exp_date,gamma,high,impliedVolatility,lastPrice,lastSize,low,open,strike,theta,vega,volume
0,NaN,NaN,NaN,NaN,NaN,0.484916,20231215,0.048099,NaN,0.185282,NaN,NaN,NaN,NaN,375.0,-0.343613,0.178969,NaN
0,-1.0,0.0,-1.0,0.0,4.50,0.506014,20231222,0.032659,5.35,0.178812,5.21,34.0,3.3,0.0,375.0,-0.228040,0.273157,1529.0
0,NaN,NaN,NaN,NaN,NaN,NaN,20231229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,375.0,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,6.75,0.526683,20240105,0.022642,7.80,0.175947,7.75,1.0,5.7,NaN,375.0,-0.162932,0.398789,1136.0


In [54]:
from ib_insync import *
import numpy as np
import pandas as pd
from datetime import time, date, datetime, timedelta
from pytz import timezone

option_chain_rec = {'exp_date':[''],'strike':[np.nan],
                'bid':[np.nan],  'bidSize':[np.nan], 
                'ask':[np.nan],  'askSize':[np.nan],                     
                'lastPrice':[np.nan], 'lastSize':[np.nan],
                'volume':[np.nan],
                'open':[np.nan], 'high':[np.nan], 'low':[np.nan], 'close':[np.nan],                 
                'impliedVolatility':[np.nan], 'delta':[np.nan], 'gamma':[np.nan],
                'vega':[np.nan], 'theta':[np.nan]}

def get_option_chain(symbol, stock_price, strike_limit_pert =0.02, min_days_to_expire=15, max_days_to_expire=30):
        
    trade_date = datetime.now(timezone('US/Eastern'))    
    
    util.startLoop()  # uncomment this line when in a notebook

    ib = IB()

    ib.connect(host, port, clientId=12345)

    x = ib.reqMatchingSymbols(symbol)
    
    conId = x[0].contract.conId
        
    x = ib.reqSecDefOptParams(symbol, "", "STK", conId)
    
    exp_date_list = list(filter(lambda x: (pd.Timestamp(x).tz_localize(timezone('US/Eastern'))-trade_date).days < max_days_to_expire, x[0].expirations))   
   
    if strike_limit_pert > 0:    
        strikes = list(filter(lambda x: x >= stock_price * (1-strike_limit_pert) and x <= stock_price * (1+strike_limit_pert), x[0].strikes))   
    else:
        strikes = list(filter(lambda x: x >= stock_price, x[0].strikes))     
        strikes = [strikes[0]]
 
    print(exp_date_list)
    
    print(strikes)
    
    #df_new_row = pd.DataFrame.from_records(self.empty_rec)

    ib.reqMarketDataType(3)
        
    df = pd.DataFrame()
    for exp_date in exp_date_list:
        for strike in strikes:        
            print(exp_date, strike)
            contract = Contract()
            contract.symbol = symbol
            contract.secType = 'OPT'
            contract.exchange = 'SMART'
            contract.currency = 'USD'
            contract.lastTradeDateOrContractMonth = exp_date
            contract.strike = strike
            contract.right = 'C'
            contract.multiplier = '100'
            ib.qualifyContracts(contract) 
            x = ib.reqMktData(contract, '', False, False, [])
            ib.sleep(5)            
            print(x)
            print(x.bid, x.ask, x.last)            
            ne = pd.DataFrame.from_records(option_chain_rec)
            ne.exp_date = exp_date
            ne.strike = strike    
            ne.bid =  x.bid
            ne.bidSize = x.bidSize
            ne.ask = x.ask
            ne.askSize = x.askSize
            ne.lastPrice = x.last
            print(ne.lastPrice, x.last)
            ne.lastSize = x.lastSize            
            ne.open = x.open
            ne.high = x.high
            ne.low = x.low
            ne.close = x.close

            m = x.modelGreeks       
            b = x.bidGreeks 
            a = x.askGreeks             
            if m != None:
                #print(m)
                ne.impliedVolatility = m.impliedVol
                ne.delta = m.delta
                ne.gamma = m.gamma
                ne.vega =  m.vega
                ne.theta = m.theta
                #ne.sigma = m.sigma
                ne.volume = x.volume

            print(ne)
            df = pd.concat([df, ne])  
                        
    ib.disconnect()
    
    return df

df = get_option_chain('MSFT', 374.23, strike_limit_pert=0)

df

['20231215', '20231222', '20231229', '20240105']
[375.0]
20231215 375.0


Error 10090, reqId 6: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.MSFT NASDAQ.NMS/TOP/ALL, contract: Contract(secType='OPT', conId=632234624, symbol='MSFT', lastTradeDateOrContractMonth='20231215', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  231215C00375000', tradingClass='MSFT')


Ticker(contract=Contract(secType='OPT', conId=632234624, symbol='MSFT', lastTradeDateOrContractMonth='20231215', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  231215C00375000', tradingClass='MSFT'), time=datetime.datetime(2023, 12, 10, 15, 14, 4, 157800, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.0, modelGreeks=OptionComputation(tickAttrib=0, impliedVol=0.1852815702270387, delta=0.4849159097275644, optPrice=3.1258953237496243, pvDividend=0.0, gamma=0.048099001444456994, vega=0.17896878285258566, theta=-0.34361349230794425, undPrice=374.3), bboExchange='c70003', snapshotPermissions=1)
nan nan nan
0   NaN
Name: lastPrice, dtype: float64 nan
   ask  askSize  bid  bidSize  close     delta  exp_date     gamma  high  \
0  NaN      NaN  NaN      NaN    NaN  0.484916  20231215  0.048099   NaN   

   impliedVolatility  lastPrice  lastSize  low  open  strike     theta  \
0           0.185282        NaN       NaN  NaN   NaN   375.0

Error 10090, reqId 8: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.MSFT NASDAQ.NMS/TOP/ALL, contract: Contract(secType='OPT', conId=663787506, symbol='MSFT', lastTradeDateOrContractMonth='20231222', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  231222C00375000', tradingClass='MSFT')


Ticker(contract=Contract(secType='OPT', conId=663787506, symbol='MSFT', lastTradeDateOrContractMonth='20231222', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  231222C00375000', tradingClass='MSFT'), time=datetime.datetime(2023, 12, 10, 15, 14, 11, 296224, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.01, bid=-1.0, bidSize=0.0, ask=-1.0, askSize=0.0, last=5.21, lastSize=34.0, volume=1529.0, open=0.0, high=5.35, low=3.3, close=4.5, ticks=[TickData(time=datetime.datetime(2023, 12, 10, 15, 14, 11, 296224, tzinfo=datetime.timezone.utc), tickType=66, price=-1.0, size=0.0), TickData(time=datetime.datetime(2023, 12, 10, 15, 14, 11, 296224, tzinfo=datetime.timezone.utc), tickType=67, price=-1.0, size=0.0)], bidGreeks=OptionComputation(tickAttrib=0, impliedVol=None, delta=None, optPrice=None, pvDividend=0.0, gamma=None, vega=-2.0, theta=-2.0, undPrice=374.3), askGreeks=OptionComputation(tickAttrib=0, impliedVol=None, delta=None, optP

Error 10090, reqId 10: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.MSFT NASDAQ.NMS/TOP/ALL, contract: Contract(secType='OPT', conId=664948989, symbol='MSFT', lastTradeDateOrContractMonth='20231229', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  231229C00375000', tradingClass='MSFT')


Ticker(contract=Contract(secType='OPT', conId=664948989, symbol='MSFT', lastTradeDateOrContractMonth='20231229', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  231229C00375000', tradingClass='MSFT'), marketDataType=3, minTick=0.01, bboExchange='c70003', snapshotPermissions=1)
nan nan nan
0   NaN
Name: lastPrice, dtype: float64 nan
   ask  askSize  bid  bidSize  close  delta  exp_date  gamma  high  \
0  NaN      NaN  NaN      NaN    NaN    NaN  20231229    NaN   NaN   

   impliedVolatility  lastPrice  lastSize  low  open  strike  theta  vega  \
0                NaN        NaN       NaN  NaN   NaN   375.0    NaN   NaN   

   volume  
0     NaN  
20240105 375.0


Error 10090, reqId 12: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.MSFT NASDAQ.NMS/TOP/ALL, contract: Contract(secType='OPT', conId=667689557, symbol='MSFT', lastTradeDateOrContractMonth='20240105', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  240105C00375000', tradingClass='MSFT')


Ticker(contract=Contract(secType='OPT', conId=667689557, symbol='MSFT', lastTradeDateOrContractMonth='20240105', strike=375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='MSFT  240105C00375000', tradingClass='MSFT'), time=datetime.datetime(2023, 12, 10, 15, 14, 17, 704564, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.01, last=7.75, lastSize=1.0, volume=1136.0, high=7.8, low=5.7, close=6.75, ticks=[TickData(time=datetime.datetime(2023, 12, 10, 15, 14, 17, 704564, tzinfo=datetime.timezone.utc), tickType=72, price=7.8, size=0.0), TickData(time=datetime.datetime(2023, 12, 10, 15, 14, 17, 704564, tzinfo=datetime.timezone.utc), tickType=73, price=5.7, size=0.0), TickData(time=datetime.datetime(2023, 12, 10, 15, 14, 17, 704564, tzinfo=datetime.timezone.utc), tickType=74, price=-1.0, size=1136.0), TickData(time=datetime.datetime(2023, 12, 10, 15, 14, 17, 704564, tzinfo=datetime.timezone.utc), tickType=75, price=6.75, size=0.0)], bidGreeks=OptionCom

,ask,askSize,bid,bidSize,close,delta,exp_date,gamma,high,impliedVolatility,lastPrice,lastSize,low,open,strike,theta,vega,volume
0,NaN,NaN,NaN,NaN,NaN,0.484916,20231215,0.048099,NaN,0.185282,NaN,NaN,NaN,NaN,375.0,-0.343613,0.178969,NaN
0,-1.0,0.0,-1.0,0.0,4.50,0.506014,20231222,0.032659,5.35,0.178812,5.21,34.0,3.3,0.0,375.0,-0.228040,0.273157,1529.0
0,NaN,NaN,NaN,NaN,NaN,NaN,20231229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,375.0,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,6.75,0.526683,20240105,0.022642,7.80,0.175947,7.75,1.0,5.7,NaN,375.0,-0.162932,0.398789,1136.0


In [57]:
df[['exp_date',  'strike', 'impliedVolatility', 'delta', 'gamma', 'vega', 'theta', 'bid', 'ask', 'lastPrice', 'volume']]

#df.columns

,exp_date,strike,impliedVolatility,delta,gamma,vega,theta,bid,ask,lastPrice,volume
0,20231215,375.0,0.185282,0.484916,0.048099,0.178969,-0.343613,NaN,NaN,NaN,NaN
0,20231222,375.0,0.178812,0.506014,0.032659,0.273157,-0.228040,-1.0,-1.0,5.21,1529.0
0,20231229,375.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,20240105,375.0,0.175947,0.526683,0.022642,0.398789,-0.162932,NaN,NaN,7.75,1136.0


In [ ]:
option_chain_rec = {'exp_date':[''],'strike':[np.nan],
                'lastPirce':[np.nan],'bid':[np.nan], 'ask':[np.nan],
                'bidSize':[np.nan], 'askSize':[np.nan], 'volume':[np.nan],
                'open':[np.nan], 'high':[np.nan], 'low':[np.nan], 'close':[np.nan],
                'impliedVolatility':[np.nan], 'delta':[np.nan], 'gamma':[np.nan],
                'vega':[np.nan], 'theta':[np.nan],'sigma':[np.nan]}
    
df = pd.DataFrame()

for exp_date in exp_date_list:
    for strike in strikes:        
        ne = pd.DataFrame.from_records(option_chain_rec)
        ne.exp_date = exp_date
        ne.strike = strike
        df = pd.concat([df, ne])
        
df

In [ ]:
import pandas as pd
import pandas_ta as ta
import pymannkendall as mk
import math
import numpy as np
from datetime import time, date, datetime, timedelta
from pytz import timezone

max_days_to_expire = 30

exp_date_list = []

exp_tbl = x.expirations        
trade_date = datetime.now(timezone('US/Eastern'))


exp_date_list = list(filter(lambda x: (pd.Timestamp(x).tz_localize(timezone('US/Eastern'))-trade_date).days < 30, x.expirations))   
    
#for exp_date in exp_tbl:    
#    exp_date = pd.Timestamp(exp_date)
#    days_to_expire = (exp_date.tz_localize(timezone('US/Eastern'))-trade_date).days

#    if days_to_expire <= 0:
#        continue

#    if days_to_expire > max_days_to_expire:
#        break

#    exp_date_list.append(exp_date)

print(exp_date_list) 

    

In [ ]:
class IBConfig:
    port = 7497
    clientId = 8888
    marketDataType = 3 #https://interactivebrokers.github.io/tws-api/market_data_type.html


In [ ]:
from ib_insync import *

class IBClient:    

    connection = None     
    
    @staticmethod
    def get_client():
        util.startLoop()  # uncomment this line when in a notebook        
        if IBClient.connection == None:
            ib = IB()
            ib.connect('127.0.0.1', IBConfig.port, clientId=IBConfig.clientId)            
            IBClient.connection = ib
            # delayed quote
            ib.reqMarketDataType(IBConfig.marketDataType)            
            return ib
        else:
            return IBClient.connection
        
    @staticmethod
    def disconnect():
        if IBClient.connection != None:
            IBClient.connection.client.disconnect()
            IBClient.connection = None     

In [ ]:
mycontract = Contract()
mycontract.symbol = "AAPL"
mycontract.secType = "STK"
mycontract.exchange = "SMART"
mycontract.currency = "USD"
        
contract = Forex('EURUSD')

ib.reqMarketDataType(3)
    
#https://interactivebrokers.github.io/tws-api/historical_bars.html#hd_what_to_show

bars = ib.reqHistoricalData(
    mycontract, endDateTime='', durationStr='1 y',
    barSizeSetting='1 day', whatToShow='TRADES', useRTH=True)

# convert to pandas dataframe:
df = util.df(bars)

df.rename(columns = {'date':'Date', 'open':'Open', 'high':'High', 'low':'Low',
                     'close':'Close', 'volume':'Volume', 'average':'Average'}, inplace = True)

print(df.set_index('Date'))

ib.disconnect()

In [ ]:
from ib_insync import *

try:
    get_ipython    
    util.startLoop()  # uncomment this line when in a notebook
except:
    pass
    
ib = IB()

symbol = 'MSFT'

ib.connect('127.0.0.1', 7497, clientId=0)

contract = Contract()
contract.symbol =  symbol
contract.secType = "OPT"
contract.exchange = "SMART"
contract.currency = "USD"

ib.reqMarketDataType(3)
    
#https://interactivebrokers.github.io/tws-api/historical_bars.html#hd_what_to_show

x = ib.reqMatchingSymbols(symbol)


conId = x[0].contract.conId

x = ib.reqSecDefOptParams(symbol, "", "STK", conId)

print(x[0])


ib.disconnect()

In [ ]:
x[0].contract.conId

In [ ]:
from ib_insync import *

try:
    get_ipython    
    util.startLoop()  # uncomment this line when in a notebook
except:
    pass
    
ib = IB()

ib.connect('127.0.0.1', port, clientId=clientID)

mycontract = Contract()
mycontract.symbol = "AAPL"
mycontract.secType = "OPT"
mycontract.exchange = "SMART"
mycontract.currency = "USD"
        
contract = Forex('EURUSD')

ib.reqMarketDataType(3)
    
#https://interactivebrokers.github.io/tws-api/historical_bars.html#hd_what_to_show

x = ib.reqSecDefOptParams("MSFT", "", "STK", 272093)

print(x[0])

ib.disconnect()

In [ ]:
#Legal interval are: 1 secs, 5 secs, 10 secs, 15 secs, 30 secs, 1 min, 
#    2 mins, 3 mins, 5 mins, 10 mins, 15 mins, 20 mins, 30 mins, 
#    1 hour, 2 hours, 3 hours, 4 hours, 8 hours, 1 day, 1W, 1M, 
                
def get_price_history(symbol, period="1 Y", interval="1 day", start=None, end=None):    

    ib = IBClient.get_client()
    
    contract = Contract()
    contract.symbol = symbol
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"

    bars = ib.reqHistoricalData(
        contract, endDateTime='', durationStr=period,
        barSizeSetting=interval, whatToShow='TRADES', useRTH=True)

    df =  util.df(bars)
    df.rename(columns = {'date':'Date', 'open':'Open', 'high':'High', 'low':'Low',
                         'close':'Close', 'volume':'Volume', 'average':'Average'}, inplace = True)
    
    IBClient.disconnect()
    
    return df.set_index('Date')

get_price_history('AAPL')
    
    


In [ ]:
apple_contract = Contract()
apple_contract.symbol = 'MSFT'
apple_contract.secType = 'STK'
apple_contract.exchange = 'SMART'
apple_contract.currency = 'USD'


#Request Market Data
app.reqMarketDataType(3)
app.reqMktData(1, apple_contract, '', False, False, [])

contract.lastTradeDateOrContractMonth = '20230519'
contract.strike = 305.0
contract.right = 'C'
contract.multiplier = '100'

In [ ]:
from ib_insync import *

util.startLoop()  # uncomment this line when in a notebook

ib = IB()

ib.connect('127.0.0.1', 7497, clientId=4)

contract = Contract()
contract.symbol = 'MSFT'
contract.secType = 'OPT'
contract.exchange = 'SMART'
contract.currency = 'USD'
contract.lastTradeDateOrContractMonth = '20230519'
contract.strike = 307.5
contract.right = 'C'
contract.multiplier = '100'
ib.qualifyContracts(contract) 
ib.reqMarketDataType(3)
x = ib.reqMktData(contract, '', False, False, [])
ib.sleep(5)

print(vars(x))

ib.disconnect()

# convert to pandas dataframe:
#df = util.df(bars)
#print(df)

In [16]:
from ib_insync import *

util.startLoop()  # uncomment this line when in a notebook

def get_ticker(symbol):
    
    ib = IB()
    ib.connect('127.0.0.1', port, clientId=clientID)

    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'

    ib.qualifyContracts(contract) 
    ib.reqMarketDataType(3)
    x = ib.reqMktData(contract, '', False, False, [])
    ib.sleep(5)

    #return x.bid, x.ask, x.last, x.bidGreeks.impliedVol, x.bidGreeks.delta, x.bidGreeks.gamma, x.bidGreeks.vega, x.bidGreeks.theta

    ib.disconnect()

    return x

x = get_ticker('AAPL')    

print(vars(x))

In [15]:
from ib_insync import *

print('1111')

util.startLoop()  # uncomment this line when in a notebook

def get_option_price(symbol, exp_date, strike, otype):
    
    print('xxx')
    ib = IB()
    ib.connect(host, port, clientId=clientID)

    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'OPT'
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    contract.lastTradeDateOrContractMonth = exp_date #'20230512'
    contract.strike = strike
    contract.right = otype
    contract.multiplier = '100'
    print(contract)
    ib.qualifyContracts(contract) 
    ib.reqMarketDataType(3)
    x = ib.reqMktData(contract, '', False, False, [])
    ib.sleep(5)
    
    return x.bid, x.ask, x.last, x.bidGreeks.impliedVol, x.bidGreeks.delta, x.bidGreeks.gamma, x.bidGreeks.vega, x.bidGreeks.theta

    ib.disconnect()

print('xxx')
get_option_price('AAPL', '20231201', 190, 'C')  
print('yyy')

In [ ]:
#https://algotrading101.com/learn/ib_insync-interactive-brokers-api-guide/

from ib_insync import *

util.startLoop()  # uncomment this line when in a notebook

ib = IB()

ib.connect()

In [ ]:
nflx_contract = Stock('NFLX', 'SMART', 'USD')

In [ ]:
ib.qualifyContracts(nflx_contract)

In [ ]:
data = ib.reqMktData(nflx_contract)

In [ ]:
data.marketPrice()

In [ ]:
eur_usd_contract = Forex('EURUSD', 'IDEALPRO')
ib.qualifyContracts(eur_usd_contract)

In [ ]:
btc_fut_contract = Future('BRR', '20231031', 'CMECRYPTO')
ib.qualifyContracts(btc_fut_contract)

In [ ]:
btc_fut_cont_contract = ContFuture('BRR', 'CMECRYPTO')
ib.qualifyContracts(btc_fut_cont_contract)

In [ ]:
ib.reqMarketDataType(3)

historical_data_nflx = ib.reqHistoricalData(
    nflx_contract, 
    '', 
    barSizeSetting='15 mins', 
    durationStr='2 D', 
    whatToShow='MIDPOINT', 
    useRTH=True
    )

In [ ]:
type(historical_data_nflx)

In [ ]:
historical_data_nflx[-1]

In [ ]:
util.df(historical_data_nflx)

In [ ]:
nflx_order = MarketOrder('BUY', 200)

In [ ]:
trade = ib.placeOrder(nflx_contract, nflx_order)

In [ ]:
def order_status(trade):
    print(trade.orderStatis.status)
    if trade.orderStatus.status == 'Filled':
        fill = trade.fills[-1]
        
        print(f'{fill.time} - {fill.execution.side} {fill.contract.symbol} {fill.execution.shares} @ {fill.execution.avgPrice}')

In [ ]:

from ib_insync import *

def order_status(trade):
    print(trade.orderStatis.status)
    if trade.orderStatus.status == 'Filled':
        fill = trade.fills[-1]
        
        print(f'{fill.time} - {fill.execution.side} {fill.contract.symbol} {fill.execution.shares} @ {fill.execution.avgPrice}')
        
util.startLoop()  # uncomment this line when in a noteboo
nflx_order = MarketOrder('BUY', 100)
msft_order = MarketOrder('BUY', 100)

nflx_trade = ib.placeOrder(nflx_contract, nflx_order)

trade.filledEvent+= order_status

In [ ]:
eurusd_bracket_order = ib.bracketOrder(
    'BUY',
    100000,
    limitPrice=1.19,
    takeProfitPrice=1.20,
    stopLossPrice=1.18
)

In [ ]:
for ord in eurusd_bracket_order:
    ib.placeOrder(eur_usd_contract, ord)

In [ ]:
visa_contract = Stock('V', 'SMART', 'USD')
ib.qualifyContracts(visa_contract)

In [ ]:
price_condition = PriceCondition(
    price=200,
    conId=visa_contract.conId,
    exch=visa_contract.exchange
    )

In [ ]:
ma_contract = Stock('MA', 'SMART', 'USD')
ib.qualifyContracts(ma_contract)

In [ ]:
from ib_insync import *

util.startLoop()  # uncomment this line when in a notebook

ib = IB()

ib.connect('127.0.0.1', 7497, clientId=123)

In [ ]:
P1S_contract = Option('MSFT', '20230519', 290, 'C', 'SMART')
ib.qualifyContracts(P1S_contract) 
P1L_contract = Option('MSFT', '20230519', 300, 'C', 'SMART')
ib.qualifyContracts(P1L_contract) 

P1S_contract.ratio = 1
P1S_contract.action = 'SELL'
P1S_contract.openClose = 0
P1S_contract.shortSaleSlot = 0
P1S_contract.exemptCode=0
P1S_contract.designatedLocation = 0   

P1L_contract.ratio = 1
P1L_contract.action = 'BUY'
P1L_contract.openClose = 0
P1L_contract.shortSaleSlot = 0
P1L_contract.exemptCode=0
P1L_contract.designatedLocation = 0   

In [ ]:
comboLegs = [P1S_contract, P1L_contract]

In [ ]:
contract = Contract(symbol='MSFT', secType='BAG', exchange='SMART', currency='USD', comboLegs=comboLegs)

In [ ]:
order = Order(action='BUY', orderType='LMT', totalQuantity=5, lmtPrice=8)

In [ ]:
trade = ib.placeOrder(contract, order)

def order_status(trade):
    print(trade.orderStatus.status)
    if trade.orderStatus.status == 'Filled':
        fill = trade.fills[-1]
        
        print(f'{fill.time} - {fill.execution.side} {fill.contract.symbol} {fill.execution.shares} @ {fill.execution.avgPrice}')
        

trade.filledEvent+= order_status

In [ ]:
trade.orderStatus


In [ ]:
trade.log

In [ ]:
ib.cancelOrder(order)

In [ ]:
trade

In [ ]:
order.orderId

In [ ]:
list = [1,2,3]

In [ ]:
list.pop(1)

In [ ]:
x = ib.portfolio(account='DU7131629')
p = util.df(x)
p

In [ ]:
x = ib.accountValues(account='')
df = util.df(x)
for index, row in df.iterrows():
    print(row['tag'], '\t----\t', row['value'])


In [ ]:
ib.managedAccounts()

In [ ]:
x = ib.accountSummary()
df = util.df(x)

for index, row in df.iterrows():
    print(row['tag'], '\t----\t', row['value'])


In [ ]:
util.df(ib.positions())

In [ ]:
util.df(ib.orders())

In [ ]:
util.df(ib.fills())

In [ ]:
dir(ib)

In [ ]:
import sys
import traceback
#sys.exc_info()
#traceback.format_exc(sys.exc_info()) 

etype, value, tb = sys.exc_info()

print(''.join(traceback.format_exception(etype, value, tb, limit)))

In [ ]:
import traceback
import sys
from pprint import pprint

#from traceback_example import produce_exception

try:
    produce_exception()
except Exception as err:
    #tb = traceback.TracebackException.from_exception(err)

    #print('Handled at stack lvl 0')
    #print(''.join(tb.stack.format()))    
    
    exc_type, exc_value, exc_tb = sys.exc_info()
    print(exc_type)
    print(exc_value)
    #print(exc_tb)
    print(traceback.format_exception(exc_type, exc_value, exc_tb))
    #print(exc_tb.tb_frame)
    #print(exc_tb.tb_lasti)
    #print(exc_tb.tb_lineno)
    #print(exc_tb.tb_next)    
    
    
    
    

In [ ]:
import traceback

try:
    1/0
except Exception as ex:
    print(ex)
    print(traceback.format_exc())
    # or
    #print(sys.exc_info()[2])    
    #x = traceback.print_exception(type(ex), ex, ex.__traceback__)
    #print(ex.__traceback__)

In [ ]:
from ib_insync import *

util.startLoop()  # only use in interactive environments (i.e. Jupyter Notebooks)

In [ ]:
ib = IB()
#ib.connect()

ib.connect("127.0.0.1", 4002, clientId=21,account='DU7131629')


In [ ]:
ib = IB()
util.logToConsole('DEBUG')
ib.connect(host='127.0.0.1', port=7497, clientId=1)

In [9]:
import sys
sys.executable

'C:\\ProgramData\\Anaconda3\\envs\\option_trader\\python.exe'